Training

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import vgg19
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio as compare_psnr, structural_similarity as compare_ssim
from torch.amp import autocast, GradScaler
from torchvision.utils import save_image
import torch.nn.functional as F

# Set memory management configuration
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Learnable Self-Attention Module
class LearnableSelfAttention(nn.Module):
    def __init__(self, channels, reduction=8):
        super(LearnableSelfAttention, self).__init__()
        self.query = nn.Conv2d(channels, channels // reduction, kernel_size=1)
        self.key = nn.Conv2d(channels, channels // reduction, kernel_size=1)
        self.value = nn.Conv2d(channels, channels, kernel_size=1)
        self.scale = nn.Parameter(torch.tensor(0.1))  # Learnable scaling parameter

    def forward(self, x):
        batch, channels, height, width = x.size()

        # Compute query, key, and value
        query = self.query(x).view(batch, -1, height * width).permute(0, 2, 1)
        key = self.key(x).view(batch, -1, height * width)
        value = self.value(x).view(batch, -1, height * width).permute(0, 2, 1)

        # Compute attention
        attention = torch.softmax(torch.bmm(query, key) / (channels ** 0.5), dim=-1)
        out = torch.bmm(attention, value).permute(0, 2, 1).view(batch, channels, height, width)

        return self.scale * out + x  # Residual connection


# Dense Residual Block
class DenseResidualBlock(nn.Module):
    def __init__(self, channels):
        super(DenseResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)

    def forward(self, x):
        out = self.relu(self.conv1(x))
        out = self.conv2(out)
        return x + out  # Residual connection


# Enhanced Generator with Dense Residual Blocks and Learnable Self-Attention
class AdvancedAttentionGenerator(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(AdvancedAttentionGenerator, self).__init__()
        self.encoder1 = nn.Sequential(nn.Conv2d(in_channels, 64, kernel_size=3, padding=1), nn.ReLU())
        self.encoder2 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1), nn.ReLU())
        self.encoder3 = nn.Sequential(nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1), nn.ReLU())

        # Latent block with dense residuals and self-attention
        self.latent = nn.Sequential(
            DenseResidualBlock(256),
            LearnableSelfAttention(256),
            DenseResidualBlock(256)
        )

        self.decoder1 = nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1)
        self.decoder2 = nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1)
        self.decoder3 = nn.Conv2d(64, out_channels, kernel_size=3, padding=1)

    def forward(self, x):
        e1 = self.encoder1(x)  # 256x256 -> 256x256
        e2 = self.encoder2(e1)  # 256x256 -> 128x128
        e3 = self.encoder3(e2)  # 128x128 -> 64x64

        latent = self.latent(e3)  # 64x64 -> 64x64

        d1 = self.decoder1(latent) + e2  # Skip connection
        d2 = self.decoder2(d1) + e1  # Skip connection
        return self.decoder3(d2)  # 256x256 -> 256x256


# Lightweight Discriminator
class LightweightDiscriminator(nn.Module):
    def __init__(self, input_nc):
        super(LightweightDiscriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(input_nc, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.InstanceNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.InstanceNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, kernel_size=4, stride=1, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=1)
        )

    def forward(self, x):
        return self.model(x)


# Dataset Class
class HazyDataset(Dataset):
    def __init__(self, hazy_dir, clear_dir, transform=None):
        self.hazy_paths = sorted([os.path.join(hazy_dir, f) for f in os.listdir(hazy_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])
        self.clear_paths = sorted([os.path.join(clear_dir, f) for f in os.listdir(clear_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])

        if len(self.hazy_paths) != len(self.clear_paths):
            raise ValueError(f"Mismatch in number of hazy ({len(self.hazy_paths)}) and clear ({len(self.clear_paths)}) images. Please check the dataset.")

        self.transform = transform

    def __len__(self):
        return len(self.hazy_paths)

    def __getitem__(self, idx):
        hazy = Image.open(self.hazy_paths[idx]).convert("RGB")
        clear = Image.open(self.clear_paths[idx]).convert("RGB")
        if self.transform:
            hazy = self.transform(hazy)
            clear = self.transform(clear)
        return hazy, clear
def hinge_loss_discriminator(real_output, fake_output):
    # Hinge loss for discriminator
    real_loss = torch.mean(torch.clamp(1 - real_output, min=0))
    fake_loss = torch.mean(torch.clamp(1 + fake_output, min=0))

    return real_loss + fake_loss

def hinge_loss_generator(fake_output):
    # Hinge loss for generator
    return -torch.mean(fake_output)

# Loss Functions
def perceptual_loss(output, target, vgg):
    output_features = vgg(output)
    target_features = vgg(target)
    return nn.L1Loss()(output_features, target_features)

# def hinge_loss(output, target):
#     # replacing 0 = -1
#     new_predicted = np.array([-1 if i==0 else i for i in output])

#     # calculating hinge loss
#     hinge_loss = np.mean([max(0, 1-x*y) for x, y in zip(target, new_predicted)])
    return hinge_loss

class SSIMLoss(nn.Module):
    def forward(self, x, y):
        x = x.detach().cpu().numpy()
        y = y.detach().cpu().numpy()

        # Ensure input shape is (H, W, C) for SSIM
        x = np.transpose(x, (0, 2, 3, 1))  # Convert (B, C, H, W) → (B, H, W, C)
        y = np.transpose(y, (0, 2, 3, 1))

        # Compute SSIM for each image in the batch
        ssim_values = [compare_ssim(x[i], y[i], data_range=1.0, win_size=3, channel_axis=-1) for i in range(x.shape[0])]

        return 1 - np.mean(ssim_values)  # Return SSIM loss


def entropy_regularization(generator_output):
    probabilities = torch.sigmoid(generator_output)  # Convert to probability distribution
    return -torch.mean(probabilities * torch.log(probabilities + 1e-8) +
                       (1 - probabilities) * torch.log(1 - probabilities + 1e-8))

def compute_total_loss(generator_output, target, discriminator, hazy, vgg, fake_output1):
    # Content Loss (L1)
    content_loss = nn.L1Loss()(generator_output, target)

    # Perceptual Loss (VGG Features)
    perceptual_loss = nn.L1Loss()(vgg(generator_output), vgg(target))

    # Discriminator outputs
    real_pair = torch.cat([hazy, target], dim=1)
    fake_pair = torch.cat([hazy, generator_output], dim=1)

    real_logits = discriminator(real_pair)
    fake_logits = discriminator(fake_pair)
    fake_output = discriminator(torch.cat([target, generator_output], dim=1))

    d_loss = hinge_loss_discriminator(fake_output1, fake_output)
    g_loss1 = hinge_loss_generator(fake_output)

    # Relativistic GAN Loss
    d_real_loss = nn.BCEWithLogitsLoss()(real_logits - torch.mean(fake_logits), torch.ones_like(real_logits))
    d_fake_loss = nn.BCEWithLogitsLoss()(fake_logits - torch.mean(real_logits), torch.zeros_like(fake_logits))
    adversarial_loss = (d_real_loss + d_fake_loss) / 2  # Balanced GAN loss

    # Structural Similarity Loss (SSIM)
    ssim_loss = SSIMLoss()(generator_output, target)

    # Entropy Regularization Loss
    entropy_loss = entropy_regularization(generator_output)

    # Final Weighted Loss Combination
    total_loss = (
        120 * content_loss +  # Slightly reduced for better perceptual learning
        100 * perceptual_loss +  # Increased to improve feature retention
        70 * ssim_loss +  # Increased to emphasize texture and fine details
        1.0 * entropy_loss +  # Increased for better regularization
        d_loss + g_loss1  # Adversarial loss remains unchanged
    )

    return total_loss




def train(generator, discriminator, train_loader, val_loader, vgg, num_epochs, checkpoint_dir, patience):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    generator.to(device)
    discriminator.to(device)
    vgg.to(device).eval()
    scaler = GradScaler()

    optimizer_G = optim.AdamW(generator.parameters(), lr=5e-5)
    optimizer_D = optim.AdamW(discriminator.parameters(), lr=1e-4)
    best_psnr = 0
    early_stop_count = 0

    for epoch in range(num_epochs):
        generator.train()
        for hazy, clear in train_loader:
            hazy, clear = hazy.to(device), clear.to(device)

            optimizer_G.zero_grad()
            optimizer_D.zero_grad()

            with autocast(device_type='cuda'):
                dehazed = generator(hazy)
                fake_output = discriminator(torch.cat([hazy, dehazed], dim=1))
                total_loss = compute_total_loss(dehazed, clear, discriminator, hazy, vgg,fake_output)  # Updated loss function

            scaler.scale(total_loss).backward()
            scaler.step(optimizer_G)
            scaler.update()

        # Validation phase
        # Validation phase
        # Validation phase
        generator.eval()
        val_psnr, val_ssim = [], []
        with torch.no_grad():
            for hazy, clear in val_loader:
                hazy, clear = hazy.to(device), clear.to(device)
                dehazed = generator(hazy)

                # Convert tensors to numpy arrays
                clear_np = clear.squeeze(0).cpu().numpy()  # Ensure (C, H, W) shape
                dehazed_np = dehazed.squeeze(0).cpu().numpy()  # Ensure (C, H, W) shape

                # Ensure array has at least 3 dimensions before transposing
                if clear_np.ndim == 3:
                    clear_np = np.transpose(clear_np, (1, 2, 0))  # Convert (C, H, W) → (H, W, C)
                    dehazed_np = np.transpose(dehazed_np, (1, 2, 0))  # Convert (C, H, W) → (H, W, C)

                # Compute PSNR
                psnr = compare_psnr(clear_np, dehazed_np, data_range=1.0)

                # Compute SSIM
                # Compute SSIM with adaptive window size
                min_dim = min(clear_np.shape[0], clear_np.shape[1])  # Find the smallest image dimension
                win_size = min(7, min_dim)  # Ensure window size does not exceed image size

                ssim = compare_ssim(clear_np, dehazed_np, data_range=1.0, channel_axis=-1, win_size=win_size)


                val_psnr.append(psnr)
                val_ssim.append(ssim)

        avg_psnr, avg_ssim = np.mean(val_psnr), np.mean(val_ssim)
        print(f"Epoch {epoch+1}/{num_epochs}, PSNR: {avg_psnr:.2f}, SSIM: {avg_ssim:.4f}")

        if avg_psnr > best_psnr:
            best_psnr = avg_psnr
            early_stop_count = 0
            torch.save(generator.state_dict(), os.path.join(checkpoint_dir, "best_generator.pth"))
        else:
            early_stop_count += 1
            if early_stop_count >= patience:
                print("Early stopping triggered.")
                return


# Paths
train_hazy_dir = "train hazy folder path"
train_clear_dir = "train clear folder path"
val_hazy_dir ="validation hazy folder path"
val_clear_dir =  "validation clear folder path"
checkpoint_dir = "checkpoint directory"
os.makedirs(checkpoint_dir, exist_ok=True)

# Dataset and DataLoader
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])
train_dataset = HazyDataset(train_hazy_dir, train_clear_dir, transform=transform)
val_dataset = HazyDataset(val_hazy_dir, val_clear_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

# Models
generator = AdvancedAttentionGenerator(3, 3)
discriminator = LightweightDiscriminator(6)
vgg = vgg19(weights="IMAGENET1K_V1").features[:16].eval()
#vgg = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=False)

# Train
train(generator, discriminator, train_loader, val_loader, vgg, num_epochs=500, checkpoint_dir=checkpoint_dir, patience=50)


Testing

In [ ]:
import os
import time  # For measuring runtime
import numpy as np
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio as compare_psnr, structural_similarity as compare_ssim
import torch
import torch.nn.functional as F
from torchvision.transforms import ToTensor

# Paths for testing set (O-Haze or NH-Haze Dataset)
test_hazy_path = "test hazy folder path"
test_clear_path = "test clear folder path"
save_dehazed_path = "folder path for saving dehazed images"

if not os.path.exists(save_dehazed_path):
    os.makedirs(save_dehazed_path)

# Helper function to load and preprocess images
def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    original_size = image.size  # Store original size (width, height)
    image = image.resize((256, 256), Image.Resampling.LANCZOS)  # Resize to 256x256 for model input
    image_tensor = ToTensor()(image).unsqueeze(0)  # Convert to tensor and add batch dim
    return image_tensor, original_size

# Helper function to save upsampled dehazed images
def save_image(image_tensor, save_path, original_size):
    image_np = image_tensor.squeeze(0).cpu().numpy().transpose((1, 2, 0))
    image_np = np.clip(image_np, 0, 1) * 255.0
    image_pil = Image.fromarray(image_np.astype('uint8'))
    image_pil = image_pil.resize(original_size, Image.Resampling.LANCZOS)  # Upsample to original size
    image_pil.save(save_path)

# Dataset Loader for Testing (aligning with training dataset loader)
class SOTSTestDataset:
    def __init__(self, hazy_dir, clear_dir):
        self.hazy_paths = sorted([os.path.join(hazy_dir, f) for f in os.listdir(hazy_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])
        self.clear_paths = sorted([os.path.join(clear_dir, f) for f in os.listdir(clear_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])

        if len(self.hazy_paths) != len(self.clear_paths):
            raise ValueError(f"Mismatch in number of hazy ({len(self.hazy_paths)}) and clear ({len(self.clear_paths)}) images. Please check the dataset.")

    def __len__(self):
        return len(self.hazy_paths)

    def __getitem__(self, idx):
        return self.hazy_paths[idx], self.clear_paths[idx]

# Load test dataset
test_dataset = SOTSTestDataset(test_hazy_path, test_clear_path)

# Initialize model and load the best checkpoint
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = AdvancedAttentionGenerator(3, 3).to(device)

# Load model checkpoint safely
checkpoint_path = "/content/best_generator.pth"
generator.load_state_dict(torch.load(checkpoint_path, map_location=device), strict=False)
generator.eval()

# Initialize metrics and runtime tracker
psnr_list = []
ssim_list = []
runtimes = []  # List to store runtime for each image

# Testing loop
for idx in range(len(test_dataset)):
    hazy_path, clear_path = test_dataset[idx]

    # Load hazy and clear images
    hazy_tensor, original_size = load_image(hazy_path)
    clear_tensor, _ = load_image(clear_path)

    hazy_tensor = hazy_tensor.to(device)
    clear_tensor = clear_tensor.to(device)

    # Measure runtime
    start_time = time.time()

    # Generate dehazed image
    with torch.no_grad():
        dehazed_tensor = generator(hazy_tensor)

    end_time = time.time()
    runtimes.append(end_time - start_time)  # Record runtime

    # Save dehazed image resized to the original size
    hazy_name = os.path.basename(hazy_path)
    save_path = os.path.join(save_dehazed_path, hazy_name)
    save_image(dehazed_tensor, save_path, original_size)

    # Resize clear and dehazed images to the original size for metric computation
    clear_resized = F.interpolate(clear_tensor, size=(original_size[1], original_size[0]), mode='bilinear', align_corners=False)
    dehazed_resized = F.interpolate(dehazed_tensor, size=(original_size[1], original_size[0]), mode='bilinear', align_corners=False)

    # Convert tensors to numpy arrays for metric computation (Ensure correct format)
    clear_np = clear_resized.squeeze(0).cpu().numpy().transpose((1, 2, 0))  # Convert to (H, W, C)
    dehazed_np = dehazed_resized.squeeze(0).cpu().numpy().transpose((1, 2, 0))  # Convert to (H, W, C)

    # Compute PSNR (Data Range should be 1.0, NOT 255, since tensors are normalized)
    psnr = compare_psnr(clear_np, dehazed_np, data_range=1.0)

    # Compute SSIM using multi-channel support
    ssim = compare_ssim(clear_np, dehazed_np, data_range=1.0, channel_axis=-1)

    # Store results
    psnr_list.append(psnr)
    ssim_list.append(ssim)

# Compute average PSNR, SSIM, and runtime
avg_psnr = np.mean(psnr_list)
avg_ssim = np.mean(ssim_list)
max_psnr = np.max(psnr_list)
max_ssim = np.max(ssim_list)
avg_runtime = np.mean(runtimes)

# Print results
print(f"Average PSNR: {avg_psnr:.4f}")
print(f"Average SSIM: {avg_ssim:.4f}")
print(f"Max PSNR: {max_psnr:.4f}")
print(f"Max SSIM: {max_ssim:.4f}")
print(f"Average Runtime per Image: {avg_runtime:.4f} seconds")
